# Part 03 - Modeling

### 01 - Import library

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import IPython.display as ipd
import librosa
import librosa.display

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten

### 02 - Unpickle the features

In [2]:
df_tr = pd.read_pickle('../pickles/train.pkl')
df_ts = pd.read_pickle('../pickles/test.pkl')
df_ho = pd.read_pickle('../pickles/hold_out.pkl')

### 03 - Transform into tensorflow-friendly format

In [4]:
# use label encoder to convert the language labels to numeric target column
encoder = LabelEncoder()
encoder.fit(df_tr['lang'])
labels = encoder.classes_
y_tr = encoder.transform(df_tr['lang'])
y_ts = encoder.transform(df_ts['lang'])
y_ho = encoder.transform(df_ho['lang'])
print(labels)

['cn' 'tw']


In [5]:
# Convert feature matrix into a keras-friendly format
X_tr = np.array(df_tr['features'].tolist())
X_tr = X_tr.reshape(X_tr.shape[0],128,256,1)
X_ts = np.array(df_ts['features'].tolist())
X_ts = X_ts.reshape(X_ts.shape[0],128,256,1)
X_ho = np.array(df_ho['features'].tolist())
X_ho = X_ho.reshape(X_ho.shape[0],128,256,1)

In [6]:
# Check the shape
print(X_tr.shape)
print(y_tr.shape)
print(X_ts.shape)
print(y_ts.shape)
print(X_ho.shape)
print(y_ho.shape)

(5924, 128, 256, 1)
(5924,)
(740, 128, 256, 1)
(740,)
(740, 128, 256, 1)
(740,)


### 04 - CNN Model

In [7]:
model = Sequential()
model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(128, 256, 1), padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (5, 5), activation='relu', padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 256, 16)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 128, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 128, 32)       12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 64, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8388736   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [9]:
model.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [10]:
EPOCHS = 100
checkpoint_filepath = '../temp_checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [11]:
history = model.fit(X_tr, y_tr, epochs=EPOCHS, validation_data=(X_ts, y_ts), callbacks=[model_checkpoint_callback])

Epoch 1/100
186/186 [==============================] - 100s 537ms/step - loss: 5.9394 - accuracy: 0.6938 - val_loss: 0.5505 - val_accuracy: 0.7095
Epoch 2/100
186/186 [==============================] - 100s 537ms/step - loss: 0.5158 - accuracy: 0.7363 - val_loss: 0.5107 - val_accuracy: 0.7649
Epoch 3/100
186/186 [==============================] - 99s 534ms/step - loss: 0.4906 - accuracy: 0.7515 - val_loss: 0.5065 - val_accuracy: 0.7784
Epoch 4/100
 46/186 [======>.......................] - ETA: 1:11 - loss: 0.4849 - accuracy: 0.7636

KeyboardInterrupt: 

In [48]:
val_loss, val_acc = model.evaluate(X_ho, y_ho, verbose=1)

24/24 [==============================] - 2s 86ms/step - loss: 0.3879 - accuracy: 0.8284


In [49]:
model.save('../saved_model/20200603')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../saved_model/20200603/assets
